# This will be a file to help determine if derviatives are correct that I derive by hand

In [1]:
import torch

from splat.utils import (
    build_rotation,
    get_extrinsic_matrix,
    getIntrinsicMatrix,
    read_camera_file,
    read_image_file,
)

scales = torch.tensor([[-5, -5.5, -4.5, -5]])
scales = torch.exp(scales)
quaternions = torch.tensor([[0.2, 0.4, 0.3, 1.0]])
point_3d = torch.tensor([[0.1, 0.1, -0.2]])
color = torch.tensor([0.3, 0.4, 0.5])
opacity = torch.tensor([0.5])


focal_x = torch.tensor([100.0])
focal_y = torch.tensor([100.0])
c_x = torch.tensor([0.0])
c_y = torch.tensor([0.0])
width = torch.tensor([99.0])
height = torch.tensor([99.0])
camera_rotation = torch.tensor([1, 0, 0, 0]).unsqueeze(0)
camera_translation = torch.tensor([[-0.1, -0.1, 0.0]])
rotation_matrix = build_rotation(camera_rotation)
extrinsic_matrix = get_extrinsic_matrix(rotation_matrix, camera_translation).T
intrinsic_matrix = getIntrinsicMatrix(
    focal_x=focal_x,
    focal_y=focal_y,
    width=width,
    height=height,
    zfar=100.0,
    znear=0.01,
    z_sign=1.0,
)

In [2]:
import math

from torch import nn

from splat.render_engine.utils import compute_fov_from_focal
from splat.utils import ndc2Pix

fovX = compute_fov_from_focal(focal_x, width)
fovY = compute_fov_from_focal(focal_y, height)

tan_fovX = math.tan(fovX / 2)
tan_fovY = math.tan(fovY / 2)

points_homogeneous = torch.cat(
    [
        point_3d,
        torch.ones(point_3d.shape[0], 1),
    ],
    dim=1,
)

def get_3d_covariance_matrix(scales, quaternions):
    quaternions = nn.functional.normalize(quaternions, p=2, dim=1)
    # nx3x3 matrix
    rotation_matrices = build_rotation(quaternions)
    # nx3x3 matrix
    scale_matrices = torch.zeros((len(point_3d), 3, 3))
    scale = torch.exp(scales)
    scale_matrices[:, 0, 0] = scale[:, 0]
    scale_matrices[:, 1, 1] = scale[:, 1]
    scale_matrices[:, 2, 2] = scale[:, 2]
    m = rotation_matrices @ scale_matrices
    covariance = m @ m.transpose(1, 2)
    return covariance

def invert_covariance_2d(covariance_2d: torch.Tensor, epsilon: float = 1e-6) -> torch.Tensor:
    """Covariance will ve a nX2X2 tensor"""
    cov = covariance_2d + epsilon
    determinant = cov[:, 0, 0] * cov[:, 1, 1] - cov[:, 0, 1] * cov[:, 1, 0]
    inverted_cov = torch.zeros_like(cov).to(cov.device)
    multiplier = 1.0 / determinant
    inverted_cov[:, 0, 0] = cov[:, 1, 1] * multiplier
    inverted_cov[:, 0, 1] = -cov[:, 0, 1] * multiplier
    inverted_cov[:, 1, 0] = -cov[:, 1, 0] * multiplier
    inverted_cov[:, 1, 1] = cov[:, 0, 0] * multiplier
    return inverted_cov

covariance_3d = get_3d_covariance_matrix(scales, quaternions)
points_camera_space  = points_homogeneous @ extrinsic_matrix
x = points_camera_space[:, 0] / points_camera_space[:, 2]
y = points_camera_space[:, 1] / points_camera_space[:, 2]
x = torch.clamp(x, -1.3 * tan_fovX, 1.3 * tan_fovX) * points_camera_space[:, 2]
y = torch.clamp(y, -1.3 * tan_fovY, 1.3 * tan_fovY) * points_camera_space[:, 2]
z = points_camera_space[:, 2]
j = torch.zeros((points_camera_space.shape[0], 3, 3))
j[:, 0, 0] = focal_x / z
j[:, 0, 2] = -(focal_x * x) / (z ** 2)
j[:, 1, 1] = focal_y / z
j[:, 1, 2] = -(focal_y * y) / (z ** 2)

w = extrinsic_matrix[:3, :3]
t = w @ j.transpose(1, 2)
covariance2d = (
    t.transpose(1, 2)
    @ covariance_3d.transpose(1, 2) # doesnt this not do anything?
    @ t
)
# scale by 0.3 for the covariance and numerical stability on the diagonal
# this is a hack to make the covariance matrix more stable
# this is a hack to make the covariance matrix more stable
covariance2d[:, 0, 0] = covariance2d[:, 0, 0] + 0.3
covariance2d[:, 1, 1] = covariance2d[:, 1, 1] + 0.3

inverted_covariance_2d = invert_covariance_2d(covariance2d)

print(intrinsic_matrix)
points_ndc = points_camera_space @ intrinsic_matrix
points_ndc[:, :2] = points_ndc[:, :2] / points_ndc[:, 3].unsqueeze(1) 
points_ndc = points_ndc[:, :3]  # nx3
pixel_coords_x = ndc2Pix(points_ndc[:, 0], dimension=width)
pixel_coords_y = ndc2Pix(points_ndc[:, 1], dimension=height)
points_ndc[:, 0] = pixel_coords_x
points_ndc[:, 1] = pixel_coords_y
print(points_ndc)
print(covariance2d)
print(inverted_covariance_2d)

tensor([[ 2.0202,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  2.0202,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0001, -0.0200],
        [ 0.0000,  0.0000,  1.0000,  0.0000]])
tensor([[49.0000, 49.0000,  0.8000]])
tensor([[[254503.1406,    408.5012,      0.0000],
         [   408.5012, 252796.6562,      0.0000],
         [     0.0000,      0.0000,      0.0000]]])
tensor([[[ 3.9292e-06, -6.3494e-09,  0.0000e+00],
         [-6.3494e-09,  3.9558e-06,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]])


In [3]:
def extract_gaussian_weight(
    pixel: torch.Tensor, mean: torch.Tensor, inv_covariance: torch.Tensor, pdb: bool = False
) -> torch.Tensor:
    """
    Use the covariance matrix to extract the weight of the point

    Args:
        mean: 1x2 tensor
        covariance: 2x2 tensor
    """
    diff = pixel - mean
    diff = diff.unsqueeze(0)
    gaussian_weight = -0.5 * torch.matmul(diff, torch.matmul(inv_covariance, diff.t()))
    actual_weight = torch.exp(gaussian_weight)
    return actual_weight, gaussian_weight

def render_pixel(
    x_value: int,
    y_value: int,
    mean_2d: torch.Tensor,
    inv_covariance_2d: torch.Tensor,
    opacity: torch.Tensor,
    color: torch.Tensor,
    current_T: float,
    min_weight: float = 0.00001,
    verbose: bool = False,
):
    """Uses alpha blending to render a pixel"""
    gaussian_strength, exponent_weight = extract_gaussian_weight(
        torch.Tensor([x_value, y_value]), mean_2d, inv_covariance_2d
    )
    alpha = gaussian_strength * torch.sigmoid(opacity)
    test_t = current_T * (1 - alpha)
    if verbose:
        print(
            f"x_value: {x_value}, y_value: {y_value}, gaussian_strength: {gaussian_strength}, alpha: {alpha}, test_t: {test_t}, mean_2d: {mean_2d}"
        )
    if test_t < min_weight:
        return
    return color * current_T * alpha, test_t, current_T, gaussian_strength, exponent_weight

def compute_gradient_wrt_ndc(
    current_T: float,
    color: torch.Tensor,
    opacity: torch.Tensor,
    gaussian_weight: torch.Tensor,
    exponent_weight: torch.Tensor,
    inverted_covariance_2d: torch.Tensor,
    diff_x: torch.Tensor,
    diff_y: torch.Tensor,
):
    difference = torch.cat([diff_x, diff_y], dim=1)
    grad = current_T * color * torch.sigmoid(opacity) * torch.exp(exponent_weight)
    grad = grad * -0.5 * (inverted_covariance_2d + inverted_covariance_2d.transpose(1, 2)) @ difference.view(-1, 2, 1) * -1
    return grad

point_mean = torch.tensor([[47.0, 49.0]]).requires_grad_(True)
# inverted_covariance_2d = torch.tensor([[1.0, 0.0, 0.0, 1.0]]).view(1, 2, 2)
opacity = torch.tensor([0.5])
color = torch.tensor([0.3, 0.4, 0.5])
pixel_coords = torch.tensor([[50.0, 50.0]])

output, test_t, current_T, gaussian_strength, exponent_weight = render_pixel(
    pixel_coords[0, 0], 
    pixel_coords[0, 1], 
    point_mean[0, :2], 
    inverted_covariance_2d[:, :2, :2], 
    opacity, 
    color, 
    1.0
)

loss = output[0, 0, 0] - 1.0
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_mean.grad)

output_ndc = compute_gradient_wrt_ndc(
    current_T=torch.Tensor([[1.0], [1.0]]),
    color=torch.Tensor([[color[0]], [color[0]]]),
    opacity=torch.Tensor([[opacity], [opacity]]),
    gaussian_weight=torch.Tensor([[gaussian_strength], [gaussian_strength]]),
    exponent_weight=torch.Tensor([[exponent_weight], [exponent_weight]]),
    inverted_covariance_2d=torch.stack([inverted_covariance_2d[:, :2, :2], inverted_covariance_2d[:, :2, :2]]).view(-1, 2, 2),
    diff_x=torch.Tensor([[pixel_coords[0, 0] - point_mean[0, 0]], [pixel_coords[0, 0] - point_mean[0, 0]]]),
    diff_y=torch.Tensor([[pixel_coords[0, 1] - point_mean[0, 1]], [pixel_coords[0, 1] - point_mean[0, 1]]]),
)
print("output_ndc gradient: ", output_ndc)

Loss:  tensor(-0.8133, grad_fn=<SubBackward0>)
Gradient:  tensor([[2.2000e-06, 7.3512e-07]])
output_ndc gradient:  tensor([[[2.2000e-06],
         [7.3512e-07]],

        [[2.2000e-06],
         [7.3512e-07]]])


In [4]:
import torch


def ndc2Pix(points: torch.Tensor, dimension: int) -> torch.Tensor:
    """
    Convert points from NDC to pixel coordinates
    """
    return (points + 1) * (dimension - 1) * 0.5

def compute_ndc_gradient(
    intrinsic_matrix: torch.Tensor,
    extrinsic_matrix: torch.Tensor,
    point_ndc_1: torch.Tensor,
    final_point_ndc: torch.Tensor,
    ground_truth_ndc: torch.Tensor,
    width: torch.Tensor,
    height: torch.Tensor,
):
    pixel_gradient = torch.tensor([[(width - 1) / 2, 0], [0, (height - 1) / 2]])
    jacobian_pndc_1 = torch.zeros((point_ndc_1.shape[0], 2, 4))
    jacobian_pndc_1[:, 0, 0] = 1/point_ndc_1[:, 3]
    jacobian_pndc_1[:, 1, 1] = 1/point_ndc_1[:, 3]
    jacobian_pndc_1[:, 0, 3] = -point_ndc_1[:, 0] / point_ndc_1[:, 3]**2
    jacobian_pndc_1[:, 1, 3] = -point_ndc_1[:, 1] / point_ndc_1[:, 3]**2
    two_by_4_jacobian = pixel_gradient @ jacobian_pndc_1 @ intrinsic_matrix.T @ extrinsic_matrix.T
    print("two_by_4_jacobian: ", two_by_4_jacobian)
    derivative_l2_wrt_pndc = torch.zeros((point_ndc_1.shape[0], 1, 2))
    print("final_point_ndc: ", final_point_ndc)
    print("ground_truth_ndc: ", ground_truth_ndc)
    derivative_l2_wrt_pndc[:, :, 0] = (final_point_ndc[:, :1] - ground_truth_ndc[:, :1]) * 2
    derivative_l2_wrt_pndc[:, :, 1] = (final_point_ndc[:, 1:2] - ground_truth_ndc[:, 1:2]) * 2
    print("derivative_l2_wrt_pndc: ", derivative_l2_wrt_pndc)
    final_grad = derivative_l2_wrt_pndc @ two_by_4_jacobian
    return final_grad[:, :, :3]

point_3d = torch.tensor([[0.10001, 0.10001, -0.2]]).requires_grad_(True)
homogeneous_point = torch.cat([point_3d, torch.ones(point_3d.shape[0], 1)], dim=1)
point_camera_space = homogeneous_point @ extrinsic_matrix
point_ndc_1 = point_camera_space @ intrinsic_matrix

point_ndc = point_ndc_1[:, :2] / point_ndc_1[:, 3].unsqueeze(1)
pixel_space_x = ndc2Pix(point_ndc[:, 0], width)
pixel_space_y = ndc2Pix(point_ndc[:, 1], height)
final_point_ndc = torch.cat([pixel_space_x, pixel_space_y]).view(-1, 2)
ground_truth_ndc = torch.tensor([[48.0, 52.0]])

print(final_point_ndc - ground_truth_ndc)
loss = (final_point_ndc - ground_truth_ndc).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_3d.grad)

computed_gradient = compute_ndc_gradient(
    intrinsic_matrix=intrinsic_matrix,
    extrinsic_matrix=extrinsic_matrix,
    point_ndc_1=point_ndc_1,
    final_point_ndc=final_point_ndc,
    ground_truth_ndc=ground_truth_ndc,
    width=width,
    height=height,
)
print("Computed Gradient: ", computed_gradient)

tensor([[ 1.2474, -2.7526]], grad_fn=<SubBackward0>)
Loss:  tensor(9.1328, grad_fn=<SumBackward0>)
Gradient:  tensor([[ 6.1735e+04, -1.3623e+05, -3.7240e+00]])
two_by_4_jacobian:  tensor([[[ 2.4745e+04,  0.0000e+00,  1.2371e+00, -2.4745e+03],
         [ 0.0000e+00,  2.4745e+04,  1.2371e+00, -2.4745e+03]]],
       grad_fn=<UnsafeViewBackward0>)
final_point_ndc:  tensor([[49.2474, 49.2474]], grad_fn=<ViewBackward0>)
ground_truth_ndc:  tensor([[48., 52.]])
derivative_l2_wrt_pndc:  tensor([[[ 2.4948, -5.5052]]], grad_fn=<CopySlices>)
Computed Gradient:  tensor([[[ 6.1735e+04, -1.3623e+05, -3.7240e+00]]], grad_fn=<SliceBackward0>)


In [5]:
import torch


def ndc2Pix(points: torch.Tensor, dimension: int) -> torch.Tensor:
    """
    Convert points from NDC to pixel coordinates
    """
    return (points + 1) * (dimension - 1) * 0.5

def compute_ndc_gradient(
    intrinsic_matrix: torch.Tensor,
    extrinsic_matrix: torch.Tensor,
    point_ndc_1: torch.Tensor,
    final_point_ndc: torch.Tensor,
    ground_truth_ndc: torch.Tensor,
    width: torch.Tensor,
    height: torch.Tensor,
):
    pixel_gradient = torch.tensor([[(width - 1) / 2, 0], [0, (height - 1) / 2]])
    jacobian_pndc_1 = torch.zeros((point_ndc_1.shape[0], 2, 4))
    jacobian_pndc_1[:, 0, 0] = 1/point_ndc_1[:, 3]
    jacobian_pndc_1[:, 1, 1] = 1/point_ndc_1[:, 3]
    jacobian_pndc_1[:, 0, 3] = -point_ndc_1[:, 0] / point_ndc_1[:, 3]**2
    jacobian_pndc_1[:, 1, 3] = -point_ndc_1[:, 1] / point_ndc_1[:, 3]**2
    two_by_4_jacobian = pixel_gradient @ jacobian_pndc_1 @ intrinsic_matrix.T @ extrinsic_matrix.T
    derivative_l2_wrt_pndc = torch.zeros((point_ndc_1.shape[0], 1, 2))
    derivative_l2_wrt_pndc[:, :, 0] = (final_point_ndc[:, :1] - ground_truth_ndc[:, :1]) * 2
    derivative_l2_wrt_pndc[:, :, 1] = (final_point_ndc[:, 1:2] - ground_truth_ndc[:, 1:2]) * 2
    print("two_by_4_jacobian: ", two_by_4_jacobian.shape)
    print("derivative_l2_wrt_pndc: ", derivative_l2_wrt_pndc.shape)
    final_grad = derivative_l2_wrt_pndc @ two_by_4_jacobian
    return final_grad[:, :, :3]

point_3d = torch.tensor([[0.10001, 0.10001, -0.2], [0.09999, 0.10001, -0.2]]).view(-1, 3).requires_grad_(True)
homogeneous_point = torch.cat([point_3d, torch.ones(point_3d.shape[0], 1)], dim=1)
point_camera_space = homogeneous_point @ extrinsic_matrix
point_ndc_1 = point_camera_space @ intrinsic_matrix

point_ndc = point_ndc_1[:, :2] / point_ndc_1[:, 3].unsqueeze(1)
pixel_space_x = ndc2Pix(point_ndc[:, 0], width)
pixel_space_y = ndc2Pix(point_ndc[:, 1], height)
final_point_ndc = torch.cat([pixel_space_x.view(-1, 1), pixel_space_y.view(-1, 1)], dim=1)
ground_truth_ndc = torch.tensor([[48.0, 52.0], [48.0, 52.0]])

loss = (final_point_ndc - ground_truth_ndc).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_3d.grad)

computed_gradient = compute_ndc_gradient(
    intrinsic_matrix=intrinsic_matrix,
    extrinsic_matrix=extrinsic_matrix,
    point_ndc_1=point_ndc_1,
    final_point_ndc=final_point_ndc,
    ground_truth_ndc=ground_truth_ndc,
    width=width,
    height=height,
)
print("Computed Gradient: ", computed_gradient)

Loss:  tensor(17.2759, grad_fn=<SumBackward0>)
Gradient:  tensor([[ 6.1735e+04, -1.3623e+05, -3.7240e+00],
        [ 3.7245e+04, -1.3623e+05, -8.6724e+00]])
two_by_4_jacobian:  torch.Size([2, 2, 4])
derivative_l2_wrt_pndc:  torch.Size([2, 1, 2])
Computed Gradient:  tensor([[[ 6.1735e+04, -1.3623e+05, -3.7240e+00]],

        [[ 3.7245e+04, -1.3623e+05, -8.6724e+00]]], grad_fn=<SliceBackward0>)


In [6]:
from splat.test.partials import pixelSpaceToPixels

point_3d = torch.tensor([[0.10001, 0.10001, -0.2], [0.09999, 0.10001, -0.2]]).view(-1, 3).requires_grad_(True)
ground_truth_ndc = torch.tensor([[48.0, 52.0], [48.0, 52.0]])
coords = pixelSpaceToPixels.apply(point_3d, width, height, intrinsic_matrix, extrinsic_matrix)
print("Coords: ", coords)
loss = (coords - ground_truth_ndc).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_3d.grad.shape)


Coords:  tensor([[49.2474, 49.2474],
        [48.7526, 49.2474]], grad_fn=<pixelSpaceToPixelsBackward>)
Loss:  tensor(17.2759, grad_fn=<SumBackward0>)
Gradient:  torch.Size([2, 3])


In [7]:
import torch

from splat.test.partials import (
    d_color_wrt_alpha,
    d_alpha_wrt_gaussian_strength,
    d_gaussian_strength_wrt_gaussian_weight,
    d_gaussian_weight_wrt_diff,
    d_diff_wrt_mean,
)


def extract_gaussian_weight(
    pixel: torch.Tensor, mean: torch.Tensor, inv_covariance: torch.Tensor, pdb: bool = False
) -> torch.Tensor:
    """
    Use the covariance matrix to extract the weight of the point

    Args:
        mean: 1x2 tensor
        covariance: 2x2 tensor
    """
    diff = pixel - mean
    diff = diff.unsqueeze(0)
    gaussian_weight = -0.5 * torch.matmul(diff, torch.matmul(inv_covariance, diff.t()))
    actual_weight = torch.exp(gaussian_weight)
    return actual_weight, gaussian_weight

def render_pixel(
    x_value: int,
    y_value: int,
    mean_2d: torch.Tensor,
    inv_covariance_2d: torch.Tensor,
    opacity: torch.Tensor,
    color: torch.Tensor,
    current_T: float,
    min_weight: float = 0.00001,
    verbose: bool = False,
):
    """Uses alpha blending to render a pixel"""
    gaussian_strength, exponent_weight = extract_gaussian_weight(
        torch.Tensor([x_value, y_value]), mean_2d, inv_covariance_2d
    )
    alpha = gaussian_strength * torch.sigmoid(opacity)
    test_t = current_T * (1 - alpha)
    if verbose:
        print(
            f"x_value: {x_value}, y_value: {y_value}, gaussian_strength: {gaussian_strength}, alpha: {alpha}, test_t: {test_t}, mean_2d: {mean_2d}"
        )
    if test_t < min_weight:
        return
    return color * current_T * alpha, test_t, current_T, gaussian_strength, exponent_weight


class pixelCoordToColor(torch.autograd.Function):
    @staticmethod
    def forward(
        ctx, 
        pixel_coords: torch.Tensor, 
        gaussian_coords: torch.Tensor, 
        color: torch.Tensor, 
        inv_covariance_2d: torch.Tensor,
        current_T: torch.Tensor,
        opacity: torch.Tensor,
    ) -> torch.Tensor:
        """
        pixel_coords: nx2 tensor
        gaussian_coords: nx2 tensor
        color: nx3 tensor
        inv_covariance_2d: nx2x2 tensor
        current_T: nx1 tensor   
        opacity: nx1 tensor
        """
        diff = pixel_coords - gaussian_coords
        diff = diff.unsqueeze(0)
        gaussian_weight = -0.5 * torch.matmul(diff, torch.matmul(inv_covariance_2d, diff.transpose(1, 2)))
        actual_weight = torch.exp(gaussian_weight)
        alpha = actual_weight * torch.sigmoid(opacity)
        ctx.save_for_backward(pixel_coords, gaussian_coords, color, inv_covariance_2d, current_T, opacity, gaussian_weight, diff)
        return color * current_T * alpha

    
    @staticmethod
    def backward(ctx, grad_output: torch.Tensor) -> torch.Tensor:
        pixel_coords, gaussian_coords, color, inv_covariance_2d, current_T, opacity, gaussian_weight, diff = ctx.saved_tensors
        derivative = d_color_wrt_alpha(color, current_T).view(-1, 3, 1)
        derivative = derivative * d_alpha_wrt_gaussian_strength(opacity)
        derivative = derivative * d_gaussian_strength_wrt_gaussian_weight(gaussian_weight)
        derivative = torch.bmm(derivative, d_gaussian_weight_wrt_diff(diff, inv_covariance_2d).transpose(1, 2))
        derivative = torch.bmm(derivative, d_diff_wrt_mean(diff.shape[0]))
        deriviative = torch.bmm(grad_output, derivative)
        return None, None, None, None, None, None
    
    
point_mean = torch.tensor([[47.0, 49.0]]).requires_grad_(True)
# inverted_covariance_2d = torch.tensor([[1.0, 0.0, 0.0, 1.0]]).view(1, 2, 2)
opacity = torch.tensor([0.5]).view(-1, 1)
color = torch.tensor([[0.3, 0.4, 0.5]]).view(-1, 3)
pixel_coords = torch.tensor([[50.0, 50.0]]).view(-1, 2)

output, test_t, current_T, gaussian_strength, exponent_weight = render_pixel(
    pixel_coords[0, 0], 
    pixel_coords[0, 1], 
    point_mean[0, :2], 
    inverted_covariance_2d[:, :2, :2], 
    opacity, 
    color, 
    1.0
)
print("Output: ", output)
expected = torch.tensor([[0.15, 0.2, 0.25]])
loss = (output - expected).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_mean.grad)

output = pixelCoordToColor.apply(pixel_coords, point_mean, color, inverted_covariance_2d[:, :2, :2], torch.tensor([1.0]), opacity)
loss = (output - expected).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_mean.grad)
print("Autograd Output: ", output)

Output:  tensor([[[0.1867, 0.2490, 0.3112]]], grad_fn=<MulBackward0>)
Loss:  tensor(0.0075, grad_fn=<SumBackward0>)
Gradient:  tensor([[8.9794e-07, 3.0004e-07]])
Loss:  tensor(0.0075, grad_fn=<SumBackward0>)
Gradient:  tensor([[8.9794e-07, 3.0004e-07]])
Autograd Output:  tensor([[[0.1867, 0.2490, 0.3112]]], grad_fn=<pixelCoordToColorBackward>)


In [8]:
point_3d = torch.tensor([[0.10001, 0.10001, -0.2], [0.09999, 0.10001, -0.2]]).requires_grad_(True)
opacity = torch.tensor([0.5, 0.5]).view(-1, 1)
color = torch.tensor([[0.3, 0.4, 0.5], [0.3, 0.4, 0.5]]).view(-1, 3)
pixel_coord = torch.Tensor([[50.0, 50.0], [50.0, 50.0]]).view(-1, 2)
expected = torch.tensor([[0.15, 0.2, 0.25], [0.15, 0.2, 0.25]])

homogeneous_point = torch.cat([point_3d, torch.ones(point_3d.shape[0], 1)], dim=1)
point_camera_space = homogeneous_point @ extrinsic_matrix
point_ndc_1 = point_camera_space @ intrinsic_matrix

point_ndc = point_ndc_1[:, :2] / point_ndc_1[:, 3].unsqueeze(1)
pixel_space_x = ndc2Pix(point_ndc[:, 0], width)
pixel_space_y = ndc2Pix(point_ndc[:, 1], height)
final_point_ndc = torch.cat([pixel_space_x.view(-1, 1), pixel_space_y.view(-1, 1)], dim=1)

output, test_t, current_T, gaussian_strength, exponent_weight = render_pixel(
    pixel_coord[0, 0], 
    pixel_coord[0, 1], 
    final_point_ndc[0, :2], 
    inverted_covariance_2d[0:1, :2, :2], 
    opacity[0:1], 
    color[0:1], 
    1.0
)

print("Opacity: ", opacity[1:2])
print("Color: ", color[1:2])
output2, test_t2, current_T2, gaussian_strength2, exponent_weight2 = render_pixel(
    pixel_coord[1, 0], 
    pixel_coord[1, 1], 
    final_point_ndc[1, :2], 
    inverted_covariance_2d[0:1, :2, :2], 
    opacity[1:2], 
    color[1:2], 
    1.0
)

final_output = torch.cat([output, output2], dim=0).squeeze(1)
print("Final Output: ", final_output.shape)
print("Expected: ", expected)
print("final_output - expected: ", final_output - expected)
loss = (final_output - expected).pow(2).sum()
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_3d.grad)

Opacity:  tensor([[0.5000]])
Color:  tensor([[0.3000, 0.4000, 0.5000]])
Final Output:  torch.Size([2, 3])
Expected:  tensor([[0.1500, 0.2000, 0.2500],
        [0.1500, 0.2000, 0.2500]])
final_output - expected:  tensor([[0.0367, 0.0490, 0.0612],
        [0.0367, 0.0490, 0.0612]], grad_fn=<SubBackward0>)
Loss:  tensor(0.0150, grad_fn=<SumBackward0>)
Gradient:  tensor([[ 5.5686e-03,  5.6062e-03,  5.5867e-07],
        [ 9.2358e-03,  5.6002e-03, -1.8176e-07]])


In [9]:
from splat.test.partials import pixelCoordToColor, pixelSpaceToPixels

point_3d = torch.tensor([[0.10001, 0.10001, -0.2], [0.09999, 0.10001, -0.2]]).requires_grad_(True)
opacity = torch.tensor([0.5, 0.5]).view(-1, 1)
color = torch.tensor([[0.3, 0.4, 0.5], [0.3, 0.4, 0.5]]).view(-1, 3)
pixel_coord = torch.Tensor([[50.0, 50.0], [50.0, 50.0]]).view(-1, 2)
expected = torch.tensor([[0.15, 0.2, 0.25], [0.15, 0.2, 0.25]])
inverted_covariance_2d = inverted_covariance_2d.repeat(2, 1, 1)[:, :2, :2]

pixel_space = pixelSpaceToPixels.apply(point_3d, width, height, intrinsic_matrix, extrinsic_matrix)
output = pixelCoordToColor.apply(pixel_coord, pixel_space, color, inverted_covariance_2d[:, :2, :2], torch.tensor([1.0]), opacity)

loss = (output - expected).pow(2).sum()
print("Output: ", output)
print("Expected: ", expected)
print("output - expected: ", output - expected)
print("Loss: ", loss)
loss.backward()
print("Gradient: ", point_3d.grad)

Output:  tensor([[0.1867, 0.2490, 0.3112],
        [0.1867, 0.2490, 0.3112]], grad_fn=<pixelCoordToColorBackward>)
Expected:  tensor([[0.1500, 0.2000, 0.2500],
        [0.1500, 0.2000, 0.2500]])
output - expected:  tensor([[0.0367, 0.0490, 0.0612],
        [0.0367, 0.0490, 0.0612]], grad_fn=<SubBackward0>)
Loss:  tensor(0.0150, grad_fn=<SumBackward0>)
Gradient:  tensor([[ 5.5686e-03,  5.6062e-03,  5.5867e-07],
        [ 9.2358e-03,  5.6002e-03, -1.8176e-07]])
Autograd Output:  tensor([[0.1867, 0.2490, 0.3112],
        [0.1867, 0.2490, 0.3112]], grad_fn=<pixelCoordToColorBackward>)


In [10]:
inverted_covariance_2d

tensor([[[ 3.9292e-06, -6.3494e-09],
         [-6.3494e-09,  3.9558e-06]],

        [[ 3.9292e-06, -6.3494e-09],
         [-6.3494e-09,  3.9558e-06]]])